In [55]:
import pandas as pd
import numpy as np
from ast import literal_eval
from collections import OrderedDict, defaultdict
import requests
import time
import re
import random
import glob
import csv
import sys
from pandarallel import pandarallel

In [33]:
csv.field_size_limit(sys.maxsize)
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [34]:
#import os
#os.chdir('../../../../')

In [35]:
def ReadPullRequests(file):
    print(file)
    selcols = ['pr_number', 'repo_name', 'commit author name', 'commit author email', 'committer name', 'commmitter email', 'commit sha']
    try:
        df_committers_full_subset = pd.read_csv(file, usecols = selcols, engine = 'pyarrow')
    except:
        df_committers_full_subset = pd.read_csv(file, usecols = selcols, engine="python", on_bad_lines = 'warn')        
    df_committers_full_subset = df_committers_full_subset.drop_duplicates()
    return df_committers_full_subset

In [64]:
def CreateMatchDataset(df_committers_full):
    df_committers_match = pd.concat([
        df_committers_full[['commit author name', 'commit author email']].drop_duplicates().rename(
            {'commit author name': 'name', 'commit author email': 'email'}, axis = 1),
        df_committers_full[['committer name', 'committer email']].drop_duplicates().rename(
        {'committer name': 'name', 'committer email': 'email'}, axis = 1)])[['name', 'email']].drop_duplicates().dropna()

    for indiv_type in ['commit author', 'committer']:
        df_committers_full[f'{indiv_type} details'] = df_committers_full.parallel_apply(
            lambda x: x[f'{indiv_type} name'] + "_" + x[f'{indiv_type} email'] if 
            not pd.isnull(x[f'{indiv_type} name']) and not pd.isnull(x[f'{indiv_type} email']) else np.nan, axis = 1)
        df_committers_full['commit_repo'] = df_committers_full.parallel_apply(lambda x: x['commit sha'] + "_" + x['repo_name'], axis = 1)
        df_emails = df_committers_full[~df_committers_full[f'{indiv_type} name'].isna()].groupby(
            'commit author details')[['commit_repo']].agg(list)
        df_emails['commit_repo'] = df_emails['commit_repo'].parallel_apply(lambda x: random.sample(x, min(5, len(x))))
        dict_emails = df_emails.to_dict()['commit_repo']
    
        if indiv_type == 'commit author':
            df_committers_match['commit_repo'] = df_committers_match.parallel_apply(
                lambda x: dict_emails.get(x['name']+"_"+x['email'], np.nan), axis = 1)
        else:
            df_committers_match['user_type'] = df_committers_match['commit_repo'].parallel_apply(
                lambda x: 'author' if type(x) == list else 'committer')
            df_committers_match['commit_repo'] = df_committers_match.parallel_apply(
                lambda x: dict_emails.get(x['name']+"_"+x['email'], np.nan) 
                if type(x['commit_repo']) != list else x['commit_repo'], axis = 1)

    return df_committers_full, df_committers_match

In [ ]:
df_committers_full = pd.concat([ReadPullRequests(file) for file in glob.glob('drive/output/scrape/collect_commits/*.csv')])

drive/output/scrape/collect_commits/commits_pr_RedHatQE_selenium-smart-locator.csv
drive/output/scrape/collect_commits/commits_pr_asyncee_django-easy-select2.csv
drive/output/scrape/collect_commits/commits_pr_diff-match-patch-python_diff-match-patch.csv
drive/output/scrape/collect_commits/commits_pr_camptocamp_tilecloud-chain.csv
drive/output/scrape/collect_commits/commits_pr_kvesteri_intervals.csv
drive/output/scrape/collect_commits/commits_pr_mkwiatkowski_pinocchio.csv
drive/output/scrape/collect_commits/commits_pr_jrief_django-admin-sortable2.csv
drive/output/scrape/collect_commits/commits_pr_locustio_locust.csv
drive/output/scrape/collect_commits/commits_pr_mozilla_DeepSpeech.csv
drive/output/scrape/collect_commits/commits_pr_rohankapoorcom_zm-py.csv
drive/output/scrape/collect_commits/commits_pr_tomduck_pandoc-eqnos.csv
drive/output/scrape/collect_commits/commits_pr_alecthomas_voluptuous.csv
drive/output/scrape/collect_commits/commits_pr_miketheman_pytest-socket.csv
drive/output/s

/tmp/user/20506/ipykernel_4059783/1655418907.py:7: ParserWarning: Skipping line 1362: unexpected end of data

  df_committers_full_subset = pd.read_csv(file, usecols = selcols, engine="python", on_bad_lines = 'warn')


drive/output/scrape/collect_commits/commits_pr_jaraco_tempora.csv
drive/output/scrape/collect_commits/commits_pr_qnub_django-boto.csv
drive/output/scrape/collect_commits/commits_pr_fonttools_openstep-plist.csv
drive/output/scrape/collect_commits/commits_pr_illagrenan_ares_util.csv
drive/output/scrape/collect_commits/commits_pr_googleapis_python-bigtable.csv
drive/output/scrape/collect_commits/commits_pr_sarugaku_requirementslib.csv
drive/output/scrape/collect_commits/commits_pr_aio-libs_aiologstash.csv
drive/output/scrape/collect_commits/commits_pr_jpvanhal_inflection.csv
drive/output/scrape/collect_commits/commits_pr_Yupeek_django-rest-models.csv
drive/output/scrape/collect_commits/commits_pr_jaraco_keyrings.alt.csv
drive/output/scrape/collect_commits/commits_pr_djangonauts_django-hstore.csv
drive/output/scrape/collect_commits/commits_pr_coleifer_unqlite-python.csv
drive/output/scrape/collect_commits/commits_pr_ales-erjavec_anyqt.csv
drive/output/scrape/collect_commits/commits_pr_Gibb

In [ ]:
df_committers_full.rename({'commmitter email':'committer email'}, axis = 1, inplace = True)

In [52]:
df_committers_full, df_committers_match = CreateMatchDataset(df_committers_full)

In [56]:
df_committers = pd.concat([
    pd.read_parquet('../undergrad_thesis/data/merged_data/committers_info_pr.parquet')[['name','email','committer_info']],
    pd.read_parquet('../undergrad_thesis/data/merged_data/committers_info_push.parquet')[['name','email','committer_info']]])
df_committers['committer_info'] = df_committers['committer_info'].parallel_apply(
    lambda x: literal_eval(x) if type(x) == str else x if type(x) == type(None) else
    literal_eval(x.decode().replace('false','False').replace('true','True')))


In [68]:
df_committers_match = pd.merge(df_committers_match, df_committers, how = 'left')

In [85]:
#### END DELETE
ncount = 1000
df_committers_match.reset_index(drop = True, inplace = True)
indices = np.array_split(df_committers_match[df_committers_match['committer_info'].isna()].index, ncount)

for i in np.arange(1, 2, 1):
    print(f"Iter {i}")
    df_committers_match.loc[indices[i], 'committer_info'] = df_committers_match.loc[indices[i]].apply(
        lambda x: GetCommitterData(x['commit_repo'], x['user_type']) if type(x['commit_repo']) == list else np.nan, axis = 1)
    df_committers_match.to_csv('drive/output/scrape/link_committers_profile/committers_info.csv')

Iter 1
{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest/commits/commits#get-a-commit', 'status': '404'}
{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest/commits/commits#get-a-commit', 'status': '404'}
{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest/commits/commits#get-a-commit', 'status': '404'}
{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest/commits/commits#get-a-commit', 'status': '404'}


In [84]:
def GetCommitterData(commit_repo, user_type):
    success = False
    i = 0
    username = os.environ['PRIMARY_GITHUB_USERNAME']
    token = os.environ['PRIMARY_GITHUB_TOKEN']

    while (not success) and i < len(commit_repo):
        repo_info = commit_repo[i].split("_")[1]
        sha = commit_repo[i].split("_")[0]
        api_url = f"https://api.github.com/repos/{repo_info}/commits/{sha}"
        with requests.get(api_url, auth=(username,token)) as url:
            try:
                data = url.json()
                info = data[user_type]
                time.sleep(.75)
                if info != None:
                    success = True
                    return [info['login'], info['id'], info['type'], info['site_admin']]
                i+=1
            except:
                print(data)
                i+=1
    return np.nan